# Install packages

# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import os, sys
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break

    
import sys
from data_wrangling import specific_utils
from null_behaviors import sample_null_distributions
from null_behaviors import show_null_trajectory
from data_wrangling import further_processing_class


import os, sys
import warnings
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from math import pi
from matplotlib import rc
from numpy import random
from importlib import reload
import seaborn as sns

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
print("done")

## Retrieve monkey data

Note: it's better to use data aggregated across trials. See PatternsAndFeatures class. For now, we'll just use one session's data

In [ ]:
PLAYER = "monkey"
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330"
data_item = further_processing_class.FurtherProcessing(raw_data_folder_path=raw_data_folder_path)
data_item.retrieve_or_make_monkey_data()
data_item.make_or_retrieve_ff_dataframe(exists_ok=True)
data_item.find_patterns()

data_item.make_or_retrieve_all_trial_patterns(exists_ok=True)
data_item.make_or_retrieve_pattern_frequencies(exists_ok=True)
data_item.make_or_retrieve_all_trial_features(exists_ok=True)
data_item.make_or_retrieve_feature_statistics(exists_ok=True)


all_trial_patterns = data_item.all_trial_patterns
all_trial_features = data_item.all_trial_features
pattern_frequencies = data_item.pattern_frequencies
feature_statistics = data_item.feature_statistics

# Null Scenario: Sampling Distributions of Features

We can compare the null distribution to the monkey's distribution

**Median time**: we can find that an ideal observer can get to fireflies at this speed. Then we can find the median time of the ideal observer, and express the median time of the monkey as a fraction of the median time of the ideal observer

**Assumptions**: assume fireflies are fully visible (or the ideal observer has perfect memory): calculate the median time of going from one firefly to the nearest firefly (or firefly that takes least amount of time). Also assume quadratic action cost; so constant velocity



### run function

In [ ]:
reload(sample_null_distributions)

In [ ]:
all_median_time, all_median_distance, all_median_abs_angle, all_median_abs_angle_boundary, \
all_mean_time, all_mean_distance, all_mean_abs_angle, all_mean_abs_angle_boundary, all_total_time = sample_null_distributions.sample_null_distributions_func()

In [ ]:
print(np.array(feature_statistics['label for median']))

In [ ]:
all_null_distr = {'all_median_time': all_median_time,
                 'all_median_distance': all_median_distance,
                 'all_median_abs_angle': all_median_abs_angle, 
                 'all_median_abs_angle_boundary': all_median_abs_angle_boundary,
                 'all_mean_time': all_mean_time,
                 'all_mean_distance': all_mean_distance,
                 'all_mean_abs_angle':all_mean_abs_angle,
                 'all_mean_abs_angle_boundary': all_mean_abs_angle_boundary,
                 'all_total_time':all_total_time
}

Adding a line for monkey's statistic...so ugly...I can just print

In [ ]:
feature_statistics.Item

### Plot

In [ ]:
import math
null_distr = all_median_abs_angle
item_name = 'abs_angle_last_vis'
median = feature_statistics.loc[feature_statistics['item']==item_name, 'median'].item()
sns.histplot(null_distr * 180/math.pi, stat='percent')

plt.title('Distribution of Median Absolute Angle Last Seen from 1000 Samples (with 1000 trials each sample)')
plt.show()
print('Monkey:', median)

In [ ]:
null_distr = all_median_distance
item_name = 'd_last_vis'
median = feature_statistics.loc[feature_statistics['item']==item_name, 'median'].item()
sns.histplot(null_distr, stat='percent')
plt.title('Distribution of Median Distance Last Seen from 1000 Samples (with 1000 trials each sample)')
plt.show()

print('Monkey:', median)

In [ ]:
median = feature_statistics.loc[feature_statistics['item']=='t', 'median'].item()
sns.histplot(all_median_time, stat='percent')
#plt.plot([median, median], [0, 1], linewidth=2)
plt.title('Distribution of Median Time from 1000 Samples (with 1000 trials each sample)')
plt.show()

print('Monkey:', median)

# Null Scenario: Population distribution of features

## Median abs angle

In [ ]:
# Plot distribution of median abs angles of sampled ff that are in front of the monkey (have a positive y coordinate)
num_alive_ff = 10000
arena_radius = 1000
fftheta = random.rand(num_alive_ff) * pi - pi/2
ffr = np.sqrt(random.rand(num_alive_ff)) * arena_radius
ffx = np.cos(fftheta) * ffr
ffy = np.sin(fftheta) * ffr
ffxy = np.stack((ffx, ffy), axis=1)
distances_to_ff = np.linalg.norm(ffxy, axis = 1)
angles_to_boundaries = specific_utils.calculate_angles_to_ff_boundaries(fftheta, distances_to_ff)

sns.histplot(fftheta, stat='density')
plt.title('Distribution of Absolute Angles')
plt.show() 
sns.histplot(angles_to_boundaries, stat='density')
plt.title('Distribution of Absolute Angles to Boundaries')
plt.show() 


# Also print the median of the absolute angles and the medians of the absoluate angles to boundary
median_abs_angle = np.median(np.abs(fftheta))
median_abs_angle_to_boundary = np.median(np.abs(angles_to_boundaries))
print("median_abs_angle is", median_abs_angle)
print("median_abs_angle_to_boundary is", median_abs_angle_to_boundary)

## Median distance

In [ ]:
num_alive_ff = 10000
arena_radius = 1000
ffr = np.sqrt(random.rand(num_alive_ff)) * arena_radius
sns.histplot(ffr)
plt.show()
np.median(ffr)

# Null Scenario: Pattern Frequency

(['two_in_a_row', 'visible_before_last_one', 'disappear_latest', 'waste_cluster_around_target',
                                                                   'ignore_sudden_flash', 'retry_switch', 'retry_capture', 'ff_capture_rate', 'stop_success_rate'])]

## two_in_a_row

Suppose the agent can pick up all fireflies in a row. Specifically, the agent will capture all the fireflies in a cluster and then move on to the next cluster, and so on (and a cluster can be just a single ff or multiple ff). 

So I guess this question comes to the percentage of clusters in the arena 

I can use algorithm from cluster_analysis.make_point_vs_cluster 

Here we set the max distance to be 50 cm for 2 ff to be considered in the same cluster

In [ ]:
import numpy as np
from numpy.random import default_rng
from math import pi
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import AgglomerativeClustering

rng = default_rng(0)  # reproducible

max_cluster_distance = 50.0
num_samples = 100
num_alive_ff = 200
arena_radius = 1000.0

scenario_rates = []

for _ in range(num_samples):
    # Sample 200 fireflies uniformly in a disk of radius=arena_radius
    theta = rng.random(num_alive_ff) * 2 * pi
    r = np.sqrt(rng.random(num_alive_ff)) * arena_radius
    x = np.cos(theta) * r
    y = np.sin(theta) * r
    xy = np.column_stack((x, y))

    # Single-linkage clustering with a hard distance threshold
    model = AgglomerativeClustering(
        n_clusters=None,
        distance_threshold=max_cluster_distance,
        linkage="single",
        metric="euclidean",
    ).fit(xy)

    labels = model.labels_
    counts = np.bincount(labels)

     # for each cluster with n > 1 ff, we count n-1 ff to be categorized as 'two in a row'
    multi_counts = counts[counts > 1]
    two_in_a_row_rate = (multi_counts.sum() - len(multi_counts)) / num_alive_ff
    scenario_rates.append(two_in_a_row_rate)

scenario_rates = np.asarray(scenario_rates)

sns.histplot(scenario_rates, bins=30, stat="density")
plt.xlabel("Two-in-a-row scenario rate")
plt.ylabel("Density")
plt.title("Distribution over simulations")
plt.show()

print("Median rate:", float(np.median(scenario_rates)))